In [64]:
import pandas as pd
#F1: 0.725 with lengths=64 count_data=23000
#F1: 0.639 with lengths=256 сount_data=2000
dataset = pd.read_csv('DataSets/train.csv', delimiter=';')

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
...,...,...,...,...
23123,Губахинский городской округ,ЖКХ,'Город Гремячинск-— ситуация с теплом на улице...,Ненадлежащее качество или отсутствие отопления
23124,Министерство здравоохранения,Здравоохранение/Медицина,"'Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу
23125,Лысьвенский городской округ,Благоустройство,'А что творится с благоустройством дворов! Воо...,Благоустройство придомовых территорий
23126,Лысьвенский городской округ,ЖКХ,'Сообщение без текста,Ненадлежащее качество или отсутствие отопления


In [65]:
import pymorphy3
from razdel import tokenize
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")

dataset = dataset.drop_duplicates()
dataset = dataset[dataset['Текст инцидента'] != '\'Сообщение без текста']

morph = pymorphy3.MorphAnalyzer()

stopTags = ['PNCT', 'NUMB', 'UNKN', 'LATN', 'ROMN']

def preprocess_text(text):
    tokens = tokenize(text.lower())
    tokens = [morph.parse(token.text)[0].normal_form for token in tokens 
              if morph.parse(token.text)[0].normal_form not in russian_stopwords and 
              not any(tag in morph.parse(token.text)[0].tag for tag in stopTags)]
    text = " ".join(tokens) 
    return text

def get_themes_code():
    themes = dataset['Тема'].unique()
    dict = {i:themes[i] for i in range(0, len(themes))}
    return dict

themes_code = get_themes_code()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skinn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

data = dataset.iloc[:500, :]

x = data['Текст инцидента']
y = data['Тема']
y = np.array(y.replace(themes_code))

list = []

for s in x:
    s = preprocess_text(s)
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=256, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    emb = model_output.pooler_output
    list.append((emb)[0].numpy())

x_embeddings = np.asarray(list)

In [67]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_embeddings, y, test_size=0.05, random_state=8)

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print('F1: ', round(f1_score(y_test,y_pred, average='weighted'), 3))

F1:  0.115
